In [2]:
import sys
import os
import warnings
import pandas as pd
from IPython.display import display
import glob
from tqdm import tqdm_notebook,trange, tqdm 
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
warnings.simplefilter('ignore',DeprecationWarning)
# This should be fine if tqdm not updated; it is the tqdm_notebook thats crying

output_notebook()

#### MY MODULES ####
import site
site.addsitedir('../modules/') #Looking for the module differently; I know there is something more elegant
import mymodules
from bead_detection import Bead, Beads, analyze_beads_image #Explicitly calling them out

Loading BokehJS ...

Pandas version: 1.1.4
nd2 reader version: 3.2.3


In [46]:
class Test_bead_detection(object):
    """
    A very specific case to test the initial build; Will work on it to incorporate multiple unittests
    TODO: A more larger scale test to ensure non NaNs etc in pandas
    """
    def __init__(self):
        self.test_image_folder = self.get_image_folder()
#         self.test_image_fpath = self.get_image()
#         assert (os.path.isfile(os.path.join(self.test_image_fpath)))
        self.test_params = self.get_default_params()
        self.export_folder_path = self.test_params['export_folder']
        #self.imageCount = 0

    def get_image_folder(self):
        #Hardcoding the folder structure to expect
        image_folder = os.path.join(os.getcwd(),'Prex710') 
        return image_folder
    
    def get_image(self,image_file):
        image_fpath = os.path.join(self.test_image_folder,image_file)
        return image_fpath
    
    def get_default_params(self):
        ##################### 
        ## Default parameters for the Test script  Hardcoded ###
        hough_params = dict(dp=1.5,minDist=250,param1=45,param2=45,minRadius=100,maxRadius=700)
        preprocessing_params = ('median',5) #Someday going or will need to make it better
        save_status = 'both' #save_status = ['data','mask','both','None']
        export_folder_path = os.path.join(self.test_image_folder,'test_data_export')
        #Ensure that there is no export files/folders in the export directory and start clean 
        import shutil
        if os.path.isdir(export_folder_path): shutil.rmtree(export_folder_path,ignore_errors=True)
        # Export folder because it is test; Else should be sent as key:value in params in the argparse or a default
        ####################
        ## Parameters to be passed
        params = dict(hough_params=hough_params, 
                      preprocessing_params=preprocessing_params, 
                      save_status=save_status,
                      export_folder=export_folder_path)
        return params
    
    def integrated_test(self):
        '''A simple test (because I did it so many times) to check
        that there are csv and png files in the export folder;
        '''
        all_png = mymodules.get_fpaths(self.export_folder_path,'.png')
        all_csv = mymodules.get_fpaths(self.export_folder_path,'.csv')
        return (len(all_png),len(all_csv))
    
    def test_nd2file(self,f_path):
        status, test_data_frame = analyze_beads_image(f_path, **self.test_params)
        return True, test_data_frame
    
    def test_nd2folder(self):
        all_test_nd2_fpaths = mymodules.get_fpaths(self.test_image_folder,'nd2')
        all_test_nd2_fpaths.sort()
        
        tqdm_descriptor = "Processing folder : "
        tqdm_color_blue = '#0D4F8B'
        
        counter = 0
        for counter in tqdm_notebook(range(len(all_test_nd2_fpaths)), 
                                     desc=tqdm_descriptor, colour=tqdm_color_blue, position=0):
            nd2_fpath = all_test_nd2_fpaths[counter]
            status, test_dataframe = self.test_nd2file(nd2_fpath) #Just keep shuttling kwargs around
            counter+=1
        print("Completed images in folder : {0}".format(self.test_image_folder))
        return True, self.test_image_folder

In [47]:
def run_test(*args,**kwargs):
    check_status = False
    test = Test_bead_detection()
    if args[0] == 'nd2file':
        check_status, test_dataframe = test.test_nd2file(test.get_image(args[1]))
        file_count_test = test.integrated_test()
        if (file_count_test) == (2,1):
            check_status = True
        else: check_status = False
    elif args[0] == 'nd2folder':
        check_status, test_image_folder = test.test_nd2folder()
        file_count_test = test.integrated_test()
        if (file_count_test) == (32,16): # TODO do not hardcode this
            check_status = True
        else: check_status = False
    else:
        print("INVALID PARAMETER")
        return False
    
    return check_status

In [48]:
test_filename = "201015_TGNH2_Prex710_BASE_10X_200ms_flds025.nd2"
if run_test('nd2file',test_filename):
    print("SUCCESS!! ")

SUCCESS!! 


In [49]:
result = run_test('nd2folder')
if(result):
    print("Success")


Completed images in folder : C:\Users\hosfo\Documents\GitHub\beadImageAnalysis\PREX\Prex710
Success


In [7]:
# Combine all resulting CSV files into a single dataframe
# TODO - can probably do this during the analysis

def compile_results(pathname):
    results_folder_path = os.path.join(os.path.join(os.path.join(os.getcwd(),pathname),'test_data_export'),'data_frame_export')
    all_files = glob.glob(results_folder_path + '/*.csv')
    
    li = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col = None, header=0)
        li.append(df)
    results_frame = pd.concat(li, axis=0, ignore_index=True)
    
    
    simp_results_frame = results_frame[['Filename','Cy7---oCircle Intensity','Cy7---oCircle Mean Intensity','Cy7---oCircle Median Intensity','Cy7---iCircle Intensity','Cy7---iCircle Mean Intensity','Cy7---iCircle Median Intensity','Cy7---ring Intensity','Cy7---ring Mean Intensity','Cy7---ring Median Intensity']]
    initial_grouped_results = simp_results_frame.groupby(results_frame.Filename).mean()
    initial_grouped_results = initial_grouped_results.reset_index()
    
#Get average from each condition and put into own dataframe, then add all resulting averages into one dataframe
    base = initial_grouped_results[initial_grouped_results['Filename'].str.contains('BASE')]
    mean_base = pd.DataFrame(base.mean(axis=0).to_dict(),index=[base.index.values[-1]])
    mean_base.insert(0,"Condition","BASE")
    
    meoh = initial_grouped_results[initial_grouped_results['Filename'].str.contains('MEOH')]
    mean_meoh = pd.DataFrame(meoh.mean(axis=0).to_dict(),index=[meoh.index.values[-1]])
    mean_meoh.insert(0,"Condition","MeOH")
    
    pip = initial_grouped_results[initial_grouped_results['Filename'].str.contains('PIP')]
    mean_pip = pd.DataFrame(pip.mean(axis=0).to_dict(),index=[pip.index.values[-1]])
    mean_pip.insert(0,"Condition","PIP")
    
    pitc = initial_grouped_results[initial_grouped_results['Filename'].str.contains('PITC')]
    mean_pitc = pd.DataFrame(pitc.mean(axis=0).to_dict(),index=[pitc.index.values[-1]])
    mean_pitc.insert(0,"Condition","PITC")
    
    tfa = initial_grouped_results[initial_grouped_results['Filename'].str.contains('TFA')]
    mean_tfa = pd.DataFrame(tfa.mean(axis=0).to_dict(),index=[tfa.index.values[-1]])
    mean_tfa.insert(0,"Condition","TFA")
    
    grouped_results = pd.concat([mean_meoh, mean_tfa, mean_base, mean_pip, mean_pitc],axis=0)
    
    #display(grouped_results.style)
    
    plot_results(grouped_results)
    
    #output_path = os.path.join(results_folder_path,"grouped_output.csv")
    #initial_grouped_results.to_csv(output_path)
    
    #grouped_output_path = os.path.join(results_folder_path,"averaged_grouped_output.csv")
    #grouped_results.to_csv(grouped_output_path)
    

#Plot results as bar graph using Bokeh    
def plot_results(df):
    from bokeh.plotting import figure, output_file, show
    from bokeh.models import ColumnDataSource
    from bokeh.models.tools import HoverTool
    from bokeh.transform import dodge
    from bokeh.core.properties import value
    from bokeh.models import Legend
    from bokeh.io import export_png

    from bokeh.palettes import Spectral5
    from bokeh.transform import factor_cmap
    
    df.columns = ['Condition','oCircle Intensity', 'oCircle Mean Intensity','oCircle Median Intensity','iCircle Intensity', 'iCircle Mean Intensity','iCircle Median Intensity','Ring Intensity', 'Ring Mean Intensity','Ring Median Intensity']
    source = ColumnDataSource(df)
    
    conditions = source.data['Condition'].tolist()
    p = figure(x_range=conditions, title="PREX 710 Fluorescence Intensity by Incubation Conditions", plot_width=800)
    
    dodge1 = dodge('Condition', -0.25, range=p.x_range)
    dodge2 = dodge('Condition', 0.0, range=p.x_range)
    dodge3 = dodge('Condition', 0.25, range=p.x_range)
    
    p.vbar(x=dodge1, top='oCircle Mean Intensity', width = 0.2, source=source, color='#D65353')
    p.vbar(x=dodge2, top='Ring Mean Intensity', width = 0.2, source=source, color='#F7C45F')
    p.vbar(x=dodge3, top='iCircle Mean Intensity', width = 0.2, source=source, color='#28364A')
    
    legO=p.square(color='#D65353')
    legR=p.square(color='#F7C45F')
    legI=p.square(color='#28364A')
    
    legend = Legend(items=[
        ('oCircle Mean Intensity', [legO]),
        ('Ring Mean Intensity', [legR]),
        ('iCircle Mean Intensity', [legI])
    ], location = "center")
    
    p.add_layout(legend,'right')
    p.background_fill_color = None
    p.border_fill_color = None
    
    #p.legend.location='bottom'
    #p.legend.orientation='vertical'
    p.y_range.start = 0
    
    show(p)

    
    
compile_results("PREX710")